# Frequency Shifting

Explore frequency shifting via _single sideband modulation_ (SSB).
We start at the effects of _amplitude modulation_ and introduce the _Hilbert transform_ and the _analytical signal_ to develop SSB.

For more information see **DAFX - Zölzer (Ch. Modulators and demodulators)**


## Amplitude Modulation

A signal $x$ is multiplied by a harmonic function:

$$x_{AM} =  (A_c + A_m \cdot x) \cdot cos(\Omega t)$$

In [ ]:
import numpy as np
from ipywidgets import *
import IPython.display as ipd
from scipy.io import wavfile
import scipy.signal as signal
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 3]

fs, x = wavfile.read("../../samples/Toms_diner.wav")
x = x / np.abs(x.max())

plt.plot(x)
plt.title("original")
ipd.Audio(x, rate=fs)

In [ ]:
t = np.linspace(0, x.size/fs, x.size)
f = 80 # Hz
w = 2 * np.pi * f
carrier = np.cos(w * t)

plt.plot(carrier)
plt.title("carrier")

In [ ]:
A_c = 1
A_m = 0.5
x_am = (A_c + A_m * x) * carrier

plt.plot(x_am)
plt.title("modulated signal")
ipd.Audio(x_am, rate=fs)

Let's consider, what the AM does to the spectrum of a signal.
For a single sinusoid as input signal $x$, the resulting spectrum is a carrier bin and an upper and a lower side band.
These are spaced around the carrier bin by the frequency $\omega$.

In [ ]:
# let's define a function for plotting the magnitude spectrum
from scipy.fft import rfft, fft, rfftfreq, fftfreq

def plot_mag_spec(sig, fs, name="", negative=False, f_range=(20, 20000), db_range=(0, 100)):
    w = signal.hann(sig.size)
    if negative == False:
        # just positive frequencies
        freq = rfftfreq(sig.size, 1 / fs)
        mag = 20*np.log10(np.abs(rfft(sig*w)))
        plt.semilogx(freq, mag)
        plt.xlim(f_range)
    else:
        # positive and negative frequencies
        freq = fftfreq(sig.size, 1 / fs)
        mag = 20*np.log10(np.abs(fft(sig*w)))
        plt.plot(freq[1:], mag[1:]) # omit 0Hz
        plt.xlim((-f_range[1], f_range[1]))
        plt.xscale('symlog')
    plt.ylim(db_range)
    plt.xlabel("f [Hz]")
    plt.ylabel("amplitude [dB]")
    plt.title(name)
    plt.show()

# and a function to return magnitude and frequencies
def mag_spec(sig, fs, name="", negative=False):
    if negative == False:
        freq = rfftfreq(sig.size, 1 / fs)
        mag = 20*np.log10(np.abs(rfft(sig)))
    else:
        freq = fftfreq(sig.size, 1 / fs)
        mag = 20*np.log10(np.abs(fft(sig)))
    return freq, mag

In [ ]:
# interactive AM spectrum
def AM(f_carrier=1000, f=500):
    # generate signals
    fs = 44100
    dur = 0.02
    t = np.arange(0, dur, dur/fs)
    carrier = np.cos(2*np.pi*f_carrier*t)
    x = np.cos(2*np.pi*f*t)
    a_carrier = 1
    a = 0.25
    x_am = (a_carrier + a * x) * carrier
    plot_mag_spec(x_am, fs, "AM spectrum", True)

interact(AM);

This simple case shows, what happens to every frequency in the base band spectrum, which is an audio signal for instance.
What happens is, that the spectrum of every real signal is symmetrical around the frequency zero.
The amplitude modulation shifts this symmetric spectrum to the carrier frequency
($\Omega + \omega$ and $\Omega - \omega$).

Let's see, what happens to bandlimited noise as base band signal:

In [ ]:
sig_len = 1024
fs = 44100
x = np.random.normal(0, 1, sig_len)

# lowpass filter
fc = 100
sos = signal.butter(6, fc, btype='lowpass', analog=False, output='sos', fs=fs)
x = signal.sosfilt(sos, x)

plot_mag_spec(x, fs, name="bandlimited noise spectrum", db_range=(-10, 30))

In [ ]:
f_carrier = 500
t = np.linspace(0, sig_len/fs, sig_len)
carrier = np.cos(2*np.pi*f_carrier*t)
A_c = 1
A_m = 1
x_am = (A_c + A_m * x) * carrier

plot_mag_spec(x_am, fs, name="AM with baseband spectrum", db_range=(-10, 60))

So, the whole symmetric spectrum gets shifted up by the carrier frequency.
It does not look like it is symmetric because of the logarithmic frequency scale, but it is.


## The Hilbert Transform and the Analytical Signal

For defining the single sideband modulation, we first need the hilbert transform and the concept of the analytical signal.


### Hilbert Transform

The Hilbert transform is given by the convolution (from [here](https://de.wikipedia.org/wiki/Hilbert-Transformation))

$$g(x) = f(x) * \frac{1}{\pi x}.$$

The Fourier transform of the continuous Hilbert transform is given by

$$H_H(j\omega) = -j \cdot sgn(\omega)$$

This corresponds to a phase shift of $\pi/2$ or $+90^°$ for negative frequencies and of $-\pi/2$ or $-90^°$ for positive frequencies.

The discrete Hilbert transform can be done by the non causal FIR filter given by the impulse response:

$$
h[n] = \frac{1 - cos(\pi n)}{\pi n} = \begin{cases} 2/\pi n & for\: n \: odd \\ 0 & for\: n \: even \end{cases}
$$

To make the filter causal, $h[n]$ is truncated to a finite length $N$ and time shifted to causality.
Because of that, the FIR Hilbert filter requires a compensation delay of the other signals, which do not use this filter.


### Analytic Signal

The analytic signal $x_a[n]$ is a complex signal whose imaginary part is the Hilbert transform ($H(\cdot)$) of the real part according to the real signal $x[n]$:

$$x_a[n] = x[n] + jH(x[n])$$

In the continuous frequency domain, this is

$$X_a(j\omega) = X(j\omega) + sgn(j\omega) \cdot X(j\omega)$$

The analytic signal has a single sided spectrum which is not symmetric like spectra of real signals.

With the analytic signal $x_a[n]$, we can calculate

* the **amplitude envelope** of $x[n]$

* the **instantaneous phase** of $x[n]$


### Implementing the Analytical Signal

Scipy provides a function [scipy.signal.hilbert()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.hilbert.html#r37d8c8a6fd16-1) for generating the analytical signal $x_a[n]$ to the real signal $x[n]$.
Let's determine the amplitude envelope and instantaneous frequency of an amplitude-modulated signal (taken from the link above).

In [ ]:
from scipy.signal import hilbert, chirp

dur = 1
fs = 44100
samples = int(fs*dur)
t = np.arange(samples) / fs

# chirp from 20Hz to 100Hz
x = chirp(t, 20.0, t[-1], 100.0)

# apply amplitude modulation
x *= (1.0 + 0.5 * np.sin(2.0*np.pi*3.0*t) )

"The amplitude envelope is given by magnitude of the analytic signal. The instantaneous frequency can be obtained by differentiating the instantaneous phase in respect to time. The instantaneous phase corresponds to the phase angle of the analytic signal."

In [ ]:
x_a = hilbert(x)

amplitude_envelope = np.abs(x_a)
instantaneous_phase = np.unwrap(np.angle(x_a))
instantaneous_frequency = np.diff(instantaneous_phase) / (2.0*np.pi) * fs

# plot
fig = plt.figure()
ax0 = fig.add_subplot(211)
ax0.plot(t, x, label='x[n]')
ax0.plot(t, amplitude_envelope, label='envelope')
ax0.set_xlabel("time in seconds")
ax0.legend()
ax1 = fig.add_subplot(212)
ax1.plot(t[1:], instantaneous_frequency)
ax1.set_xlabel("time in seconds")
ax1.set_ylim(0.0, 120.0)

## Single Sideband Modulation (SSB)

The Hilbert transform is used to just generate the upper or the lower sideband (from DAFX - Zölzer).
Given the carrier signal $x[n]$ and the modulator signal $m[n]:$

$$USB[n] = x[n] m[n] - H(x[n]) H(m[n])$$
$$LSB[n] = x[n] m[n] + H(x[n]) H(m[n])$$

For real-time implementations, the compensation delays according to the Hilbert filter have to be applied to the untransformed signals.

Let's do SSB with bandlimited noise:

In [ ]:
# modulator m[n]
sig_len = 4096
fs = 44100
m = np.random.normal(0, 1, sig_len)
fc = 200
sos = signal.butter(6, fc, btype='lowpass', analog=False, output='sos', fs=fs)
m = signal.sosfilt(sos, m)

# carrier x[n]
f_c = 500
t = np.linspace(0, sig_len/fs, sig_len)
x = np.cos(2*np.pi*f_c*t)

# calculate the upper side band
usb = x * m - hilbert(x) * hilbert(m)
lsb = x * m + hilbert(x) * hilbert(m)

plot_mag_spec(m, fs, "modulator", negative=True, db_range=(-20, 40))
plot_mag_spec(usb, fs, "USB", negative=True, db_range=(-20, 40))
plot_mag_spec(lsb, fs, "LSB", negative=True, db_range=(-20, 40))

# TODO: Warum sind die so ähnlich? -> Mit 2 sinüsse unterschiedlicher Amplitude als Modulator versuchen
# -> eindeutige Ergebnisse
# Ist die Formel für USB und LSB richtig? Anders im Paper für Frequency shifter